In [1]:
# Rode o código se precisar instalar o snscrape
# !pip install git+https://github.com/JustAnotherArchivist/snscrape.git

# Rode esse código se precisar instalar o Pandas
# !pip install pandas

In [2]:
# Imports
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [3]:
def ScrapTweets(max_tweets: int, search_string: str):
    # Definindo o máximo de tweets
    maxTweets = max_tweets

    # Criando a lista para salvar os dados
    tweets_list = []

    # Usando o TwitterSearchScraper para fazer o scrape dos dados data e salvar em uma lista
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_string).get_items()):
        if i>maxTweets:
            break
        tweets_list.append([tweet.date,
                            tweet.replyCount,
                            tweet.retweetCount,
                            tweet.likeCount,
                            tweet.sourceLabel,
                            tweet.rawContent,
                            tweet.user.username])

    return pd.DataFrame(tweets_list, columns=['Data e Hora',
                                              'Reply Count',
                                              'Retweet Cont',
                                              'Like Count',
                                              'Source',
                                              'Texto',
                                              'Usuario'])

In [7]:
# Na string de busca você pode usar ou omitir os marcadores from e until
# Exemplo:  'rockinrio from:anitta until:2020-07-31'

tweets = ScrapTweets(10000, 'rockinrio from:anitta until:2020-07-31')

tweets.head()


,Data e Hora,Reply Count,Retweet Cont,Like Count,Source,Texto,Usuario
0,2019-10-06 02:06:18+00:00,83,291,3826,Twitter for Android,Nossa musa lendária entra daqui a pouco no Pal...,Anitta
1,2019-10-06 01:40:55+00:00,327,2216,29420,Twitter for Android,Lights 💕 #AnittanoRockInRio #RockInRio https:/...,Anitta
2,2019-10-05 22:32:30+00:00,586,978,12777,Twitter for Android,"RAINHA FAZ ASSIM, NASCE PRONTA! O que vocês ac...",Anitta
3,2019-10-05 22:03:43+00:00,572,5534,34871,Twitter for Android,"""Não custa nada sonhar alto pra caramba."" 2013...",Anitta
4,2019-10-05 21:00:42+00:00,149,677,7432,Twitter for Android,"Vai começar, Brasil! Vai ter funk no @RockInRI...",Anitta


In [ ]:
# Exportar dataframe para CSV
tweets.to_csv('text-query-tweets.csv', sep=',', index=False)